In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
# method = METHOD_MITHRAL
method = METHOD_PQ

In [3]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train.npy'
    y_train = 'y_train.npy'
    featurepath_train= 'feature_train.npy'
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'
dir_result = os.path.join(dir_result, method)


In [4]:
feedback_bits = 64
ncodebooks=64
ncentroids=128


In [5]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  PQ
running kmeans in subspace 1/64... mse / {var(X_subs), var(X)}: 0.237, 0.00501
running kmeans in subspace 2/64... mse / {var(X_subs), var(X)}: 0.297, 0.00117
running kmeans in subspace 3/64... mse / {var(X_subs), var(X)}: 0.238, 0.01
running kmeans in subspace 4/64... mse / {var(X_subs), var(X)}: 0.178, 0.02
running kmeans in subspace 5/64... mse / {var(X_subs), var(X)}: 0.23, 0.00818
running kmeans in subspace 6/64... mse / {var(X_subs), var(X)}: 0.388, 0.00209
running kmeans in subspace 7/64... mse / {var(X_subs), var(X)}: 0.236, 0.016
running kmeans in subspace 8/64... mse / {var(X_subs), var(X)}: 0.209, 0.0289
running kmeans in subspace 9/64... mse / {var(X_subs), var(X)}: 0.289, 0.00765
running kmeans in subspace 10/64... mse / {var(X_subs), var(X)}: 0.382, 0.00201
running kmeans in subspace 11/64... mse / {var(X_subs), var(X)}: 0.282, 0.0173
running kmeans in subspace 12/64... mse / {var(X_subs), var(X)}: 0.278, 0.0305
running kmeans in subspace 13/64... mse /

In [6]:
x_test = np.load(dir_train+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [7]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 5.47771379e-02 -9.00563672e-02 -4.76380110e-01 ... -1.57207802e-01
   7.20814764e-02 -1.31261259e-01]
 [-1.62650406e-01 -1.71416357e-01 -3.15062910e-01 ... -3.92871201e-01
  -6.47299271e-03 -9.05812681e-02]
 [ 3.51385251e-02 -2.06485316e-01 -1.35509834e-01 ... -1.75443649e-01
  -9.06384960e-02  5.95138781e-02]
 ...
 [ 1.23512305e-01  4.60111983e-02 -7.49917984e-01 ... -3.78843635e-01
  -1.04666084e-01 -7.79564455e-02]
 [ 2.58177131e-01 -7.29361922e-03 -3.23479444e-01 ...  1.85065255e-01
   5.40798530e-04 -6.25260994e-02]
 [-3.29578668e-01  1.35787740e-01 -3.12257379e-01 ... -2.41373301e-01
  -9.20412540e-02  1.19832493e-01]]
(32000, 256)
